# ICT for Development - Collaborative API library

## Overview

This notebook aims to compile beginner-friendly documentation of APis relevant to the work of civic actors active in the field of international development. The project was started in 2024 by [Cedric Lombion](https://civicliteraci.es) to support his teaching at the University of Turin's ICT for Development and Social Good master's programme. Nonetheless, the project bears no formal affiliation with the university. 

Each entry aims to document key information and examples about the API being documented, with a focus on making them more accessible to beginners to APIs and command line scripting.

We recommend viewing this notebook from the [nbsanity viewer](https://nbsanity.com/static/5cde5825c56c8bc90bf503ed1e340ca0/api_testbench.html) for better navigation.

## Contributing

### Main Steps

1. Fork the repository
2. add your API description and examples
3. Ensure that your examples are well documented an suitable for beginners
4. send a pull request which will be reviewed for addition.

### Documentation Rules

1. All description items should be filled. 
2. The "use cases" descriptor should include examples relevant to international development projects
3. Up to 5 examples should be provided, each with a small introduction, with a focus on the examples relevant to international development

| Property                | Description                                                        | Possible Values |
|-------------------------|--------------------------------------------------------------------|-----------------|
| Main Website            | The primary website for the API                                    |                 |
| API Documentation       | Links to detailed API documentation                                |                 |
| Purpose                 | The main purpose or functionality of the API                       |                 |
| Example Use Cases       | Practical examples of how the API can be used                      |                 |
| Free API Calls          | Whether the API offers free calls                                  |Yes with API / Yes without API|
| Rate Limits for Free Usage | The rate limit for free API usage                               |                 |
| Other Limitations       | Other potential limitations not covered by the free usage details  |                 |



### Script writing

1. The code should be commented and structured for readbility
2. curl with --data-urlencode is the expected command for all queries. The use of alternatives should be explained.
3. If the API example output is too long, use the ```jq_outliner``` variable to produce a summarized output.

For suggestions or improvements, open an issue on the repository. Alternatively, contact us at contact [at] civicliteraci.es

In [35]:
# outliner function to generate informative summaries of verbose API outputs. 
# note that the order of the top level json objects will not be preserved

jq_outline='
# Collect all unique paths as arrays of segments
[ path(..) | map(if type == "number" then "[]" else tostring end) | join(".") ] | unique | map(split("."))

# Group paths by the first segment, filtering out any that do not have a second segment
| group_by(.[0]) | map(select(.[0][0] != null))

# Convert groups to objects with proper handling of potentially null keys
| map({
  key: (.[0][0] // "undefined_key"),  # Use 'undefined_key' or another placeholder if the key is null
  value: (map(.[1:]) | map(join(".")) | map(select(length > 0) | ".\(.)") | unique)
})

# Merge all objects into a single object, handling null keys properly
| from_entries
'

## Genderize API
* **main website**: https://genderize.io
* **API docs**: https://genderize.io/documentation
* **Purpose:** predicts the gender of an individual based on their name and (optionally) country
* **Example use cases:**
    * Disaggregate past research data by gender
    * Evaluate potential bias in projects that do not collect gender information
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: 100 names/day & 10 names per batch request
* **Other Limitations:** 
    * binary gender model
    * using localization on a batch request will apply the given country_id to every name in the batch


### Basic example

In [8]:
# We use the option --data-urlencode to make it easy to include names with special characters
curl -G "https://api.genderize.io/" --data-urlencode "name=Marie Anne" --data-urlencode "country=FR"

{"count":481613,"name":"Marie Anne","gender":"female","probability":1.0}


### Multiple names in a request

A loop can be used to construct a request that includes multiple names drawn from an array.  
the API accepts only up to 10 names.  
Adding a localization will apply to all names in the request.

In [13]:
# Defining our arrays

name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# Start building the curl command
request="curl -s -G \"https://api.genderize.io?\""

# Add each name to the curl command with --data-urlencode
for name in "${name_list[@]}"
do
    request+=" --data-urlencode \"name[]=${name}\""
done

# Check that the command is properly formed
echo "$request"

# Excute the command
bash -c "$request" | jq

curl -s -G "https://api.genderize.io?" --data-urlencode "name[]=yamen" --data-urlencode "name[]=lawson" --data-urlencode "name[]=maria" --data-urlencode "name[]=alejandro"
[
  {
    "count": 814,
    "name": "lawson",
    "gender": "male",
    "probability": 0.88
  },
  {
    "count": 2687884,
    "name": "maria",
    "gender": "female",
    "probability": 1
  },
  {
    "count": 386853,
    "name": "alejandro",
    "gender": "male",
    "probability": 1
  }
]


### Automate name and country requests and CSV export

We can create a loop that makes it possible to automate requests with both the name and country included.  
We will process the returned json with jq to transform the results into tabular data.

In [14]:
# we declare our arrays
name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# we count and store the number of values in our array to reuse it in the loop
length=${#country[@]}

# we loop through the arrays
for ((i=0; i<length; i++))
do
    curl -s -G "https://api.genderize.io/" \
    --data-urlencode "name=${name_list[$i]}" \
    --data-urlencode "country_id=${country[$i]}" |
    jq '[.name, .gender] | @csv'
done

"\"yamen\",\"male\""
"\"lawson\",\"male\""
"\"maria\",\"female\""
"\"alejandro\",\"male\""


## SerpAPI
* **main website**: https://serpapi.com/
* **API docs**: 
    * https://serpapi.com/
    * https://serpapi.com/playground
    * https://serpapi.com/search-api
* **Purpose**: getting JSON formatted search results from multiple search engines
* **Example use cases**:
    * Monitoring the news on specific topics to improve organisational response in areas with no field staff
    * Monitoring the impact of campaign communication efforts
    * Monitor Google News for reporting on corruption cases
    * Generate a list of amenities (e.g. doctor office) in an area to evaluate accessibility
* **Free API calls**: Yes, with API key
* **Rate limiting for free usage**: 100 calls / month
* **Other Limitations:** 

### Simple query filtered with jq

Example of a basic Google search query, as seen on their front page (only query that resolves without an API key).  
The output is summarized with the outliner function.

In [49]:
curl -s "https://serpapi.com/search.json" \
  --get \
  --data-urlencode "q=Coffee" \
  --data-urlencode "location=Austin, Texas, United States" \
  --data-urlencode "hl=en" \
  --data-urlencode "gl=us" \
  --data-urlencode "google_domain=google.com" \
  | jq "$jq_outline"

{
  "immersive_products": [
    ".[]",
    ".[].extensions",
    ".[].extensions.[]",
    ".[].extracted_original_price",
    ".[].extracted_price",
    ".[].immersive_product_page_token",
    ".[].original_price",
    ".[].price",
    ".[].serpapi_link",
    ".[].source",
    ".[].thumbnail",
    ".[].title"
  ],
  "local_map": [
    ".gps_coordinates",
    ".gps_coordinates.altitude",
    ".gps_coordinates.latitude",
    ".gps_coordinates.longitude",
    ".image",
    ".link"
  ],
  "local_results": [
    ".more_locations_link",
    ".places",
    ".places.[]",
    ".places.[].address",
    ".places.[].description",
    ".places.[].lsig",
    ".places.[].place_id",
    ".places.[].place_id_search",
    ".places.[].position",
    ".places.[].rating",
    ".places.[].reviews",
    ".places.[].reviews_original",
    ".places.[].service_options",
    ".places.[].service_options.dine_in",
    ".places.[].thumbnail",
    ".places.[].title",
    ".places.[].type"
  ],
  "organic_results": [

### Filtered data and export to CSV

Same basic query, but with an explicit attribute filter and csv conversion with jq

In [39]:
curl -s "https://serpapi.com/search.json" \
  --get \
  --data-urlencode "q=Coffee" \
  --data-urlencode "location=Austin, Texas, United States" \
  --data-urlencode "hl=en" \
  --data-urlencode "gl=us" \
  --data-urlencode "google_domain=google.com"\
  | jq '.organic_results[] | [.title,.link, .snippet] | @csv'

"\"Coffee\",\"https://en.wikipedia.org/wiki/Coffee\",\"Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due ...\""
"\"Starbucks Coffee Company\",\"https://www.starbucks.com/\",\"More than just great coffee. Explore the menu, sign up for Starbucks® Rewards, manage your gift card and more.\""
"\"Coffee | Origin, Types, Uses, History, & Facts\",\"https://www.britannica.com/topic/coffee\",\"Coffee, beverage brewed from the roasted and ground seeds of the tropical evergreen coffee plant of African origin. Coffee is one of the ...\""
"\"What is Coffee?\",\"https://www.ncausa.org/About-Coffee/What-is-Coffee\",\"Coffee traces its origin to a genus of plants known as Coffea. Within the genus there are over 500 genera and 6,000 species of tropical trees and shrubs.\""
"\"Coffee | The Nutrition Source - hsph.harvard.edu\",\"https://www.hsph.harvard.edu/nutritionsource/food-features/coffee/\",\"

## Country Info API

* **main website**: https://countryinfoapi.com
* **API docs**: https://countryinfoapi.com
* **Purpose**: fetch additional info about a country/continent etc
* **Example use cases**: 
    * adding contextual geographical information to a list of cities or countries
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: unknown
* **Other Limitations:**



### Loop example

Using a loop to get additional info about a list of countries.

In [81]:
# the api structure doesn't use a typical REST format, so we use jq's uri encoding
# instead of --data-urlencode

countries=("France" "Togo" "Italy" "Germany" "Syria" "Ivory Coast" "New Zealand")

for i in "${countries[@]}"
do  
    country=$(echo -n "$i" | jq -sRr @uri)
    curl -s "https://countryinfoapi.com/api/countries/name/$country" | jq '[.name,.region,.population] | @csv'
done


"\"France\",\"Europe\",67391582"
"\"Togo\",\"Africa\",8278737"
"\"Italy\",\"Europe\",59554023"
"\"Germany\",\"Europe\",83240525"
"\"Syria\",\"Asia\",17500657"
"\"Ivory Coast\",\"Africa\",26378275"
"\"New Zealand\",\"Oceania\",5084300"


## Open Meteo API

* **main website**: https://api.open-meteo.com
* **API docs**: https://open-meteo.com/en/docs
* **Purpose**: get historical weather data, weather forecast data, marine weather data and climate change-related data
* **Example use cases**: 
    * monitor flood risks related to heavy rains
    * Monitor weather to adjust field deployments
    * Compare how different locations are affected by climate change
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: 10,000 /day
* **Other Limitations:**

### Basic example

Summarized wth the outliner function.

In [58]:
curl -s "https://api.open-meteo.com/v1/forecast" \
  --get \
  --data-urlencode "latitude=52.5200" \
  --data-urlencode "longitude=13.4050" \
  --data-urlencode "hourly=precipitation" \
  --data-urlencode "start_date=2024-04-10" \
  --data-urlencode "end_date=2024-04-11" \
  --data-urlencode "timezone=auto" \
| jq "$jq_outline"

{
  "elevation": [],
  "generationtime_ms": [],
  "hourly": [
    ".precipitation",
    ".precipitation.[]",
    ".time",
    ".time.[]"
  ],
  "hourly_units": [
    ".precipitation",
    ".time"
  ],
  "latitude": [],
  "longitude": [],
  "timezone": [],
  "timezone_abbreviation": [],
  "utc_offset_seconds": []
}


### Calculating daily precipitations

Relevant for analysis around flood risks.  
Same query as before, but the timeframe is extened over 5 days instead of 24h

In [59]:
curl -s "https://api.open-meteo.com/v1/forecast" \
  --get \
  --data-urlencode "latitude=52.5200" \
  --data-urlencode "longitude=13.4050" \
  --data-urlencode "hourly=precipitation" \
  --data-urlencode "start_date=2024-04-10" \
  --data-urlencode "end_date=2024-04-15" \
  --data-urlencode "timezone=auto" \
| jq '[.hourly.precipitation[]] | add'

0.9000000000000001


## Healthsites.io

* **main website**: https://healthsites.io
* **API docs**: https://healthsites.io/api/docs/
* **Purpose**: get structured data about health amenities from OpenStreetMap
* **Example use cases**: 
    * measure health infrastructure accessibility
* **Free API calls**: Yes, with API key
* **Rate limiting for free usage**: unknown
* **Other Limitations:**

## Nominatim

* **main website**: https://nominatim.org/
* **API docs**: https://nominatim.org/release-docs/develop/api/Overview/
* **Purpose**: Geocoding and reverse geocoding
* **Example use cases**: 
    * find the coordinates of specific addresses or amenities
    * find an address based on its coordinates
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: 1 request per second
* **Other Limitations:**
    * Usage policy: https://operations.osmfoundation.org/policies/nominatim/ 

Reliefweb

In [89]:
APPNAME="your_app_name"  # Replace with your actual ReliefWeb app name
DISASTER_TYPES=("earthquake" "flood")  # Disaster types array
LIMIT=5  # Results limit for simplicity

BASE_URL="https://api.reliefweb.int/v1/reports"

for DISASTER in "${DISASTER_TYPES[@]}"; do
    echo "Fetching general reports for $DISASTER..."
    REQUEST_URL="${BASE_URL}?appname=${APPNAME}&query[value]=${DISASTER}&limit=${LIMIT}"
    echo "URL: $REQUEST_URL"  # Print the request URL

    RESPONSE=$(curl -s "$REQUEST_URL")
    
    if [[ $(echo $RESPONSE | jq '.data | length') -eq 0 ]]; then
        echo "No general data found for $DISASTER."
    else
        echo "Data found for $DISASTER, refining search to include funding themes..."
        # Now refine the search to include funding
        FUNDING_URL="${REQUEST_URL}&filter[field]=theme.name&filter[value]=funding"
        FUNDING_RESPONSE=$(curl -s "$FUNDING_URL")
        if [[ $(echo $FUNDING_RESPONSE | jq '.data | length') -eq 0 ]]; then
            echo "No funding specific data found for $DISASTER."
        else
            echo $FUNDING_RESPONSE | jq '.data[] | {title: .fields.title, url: .fields.url, country: .fields.primary_country.name, date: .fields.date.created}'
        fi
    fi
done


Fetching general reports for earthquake...
URL: https://api.reliefweb.int/v1/reports?appname=your_app_name&query[value]=earthquake&limit=5
No general data found for earthquake.
Fetching general reports for flood...
URL: https://api.reliefweb.int/v1/reports?appname=your_app_name&query[value]=flood&limit=5
No general data found for flood.
